# Mesurer une distance avec un microcontrôleur Arduino

## 1. Le montage
Le montage est très simple.  
On relie le trigger au connecteur 7 et l'echo au connecteur 6 sans oublier l'alimentation 5V et masse (Ground)
![montage](telemetre.JPG)

## 2. Le principe
En mettant la sortie 7 au niveau haut pendant 10µs, on provoqie l'émission d'une salve d' ultrasons de durée 10µs.
Lorsque l'écho de cette onde est reçu par le capteur, la broche echo passe au niveau haut.
Il suffit donc de mesurer la durée écoulée depuis l'émission de la salve pour connaître avec l'instruction _pulseIn(DIN_ECHO, HIGH)_
Attention, celle-ci est en microseconde.  
En prenant l'hypothèse que la célérité des US est 340 m/s,  
il vient d = v . $\Delta$t. $10^{-6}$ /2 soit d = 340*$10^{-6}$/2 = 1,7.$10^{-4}$.$\Delta$t.   
Si on veut le résultat en cm, il faut encore multiplier par 100, ce qui donne : d= $\frac{17}{1000}$.$\Delta$t.  
On remarque ausi que $\frac{1000}{17}$ = 58,8235 donc on peut simplement calculer d avec la formule :  

\begin{align}
\fbox{ d = pulseIn(DIN_ECHO, HIGH)/58.8235 }
\end{align}


## 3. Le code Arduino
### A n'exécuter que sur un arduino


In [ ]:

const char DOUT_TRIGGER = 7;
const char DIN_ECHO = 6;
float distance;

void setup() {
    //On configure le trigger en sortie et l écho en entrée
    pinMode(DIN_ECHO, INPUT);
    pinMode(DOUT_TRIGGER, OUTPUT);
    //on initialise le moniteur série
    Serial.begin(9600)
}

void loop() {

//emission salve US de 10µs
    digitalWrite(DOUT_TRIGGER, LOW);
    delayMicroseconds(2);
    digitalWrite(DOUT_TRIGGER, HIGH);
    delayMicroseconds(10);
    digitalWrite(DOUT_TRIGGER, LOW);
   
//reception echo
    distance= pulseIn(DIN_ECHO, HIGH) /58.8235;
    //d=(34000*t/1000000)/2=17/1000*t (= t/58.8235)
    
    //le télémetre est limité à 4 mètres    
    if (distance>400){
        Serial.println("echec de la mesure");
        delay(800);
    }
    else{
        Serial.println("distance "+distance+" cm");
        delay(2000);
    }
}

## 4. La photo du montage avec un afficheur LCD
![photo](photo_telemetre.jpg)

## 5. Le programme gérant l'afficheur

In [ ]:
#include <LiquidCrystal_I2C.h>
#include <Wire.h>
LiquidCrystal_I2C lcd(0x20,16,2);//0x3F est a changé en fonction de LCD
const char DOUT_TRIGGER = 7;
const char DIN_ECHO = 6;
float distance;

void setup() {
    lcd.init();
    lcd.backlight();
    lcd.setCursor(0,0);
    lcd.print("Bonjour");
    delay(2000);
    lcd.setCursor(0,1);
    lcd.print("debut mesure");
    pinMode(DIN_ECHO, INPUT);
    pinMode(DOUT_TRIGGER, OUTPUT);
    delay(1000);
}

void loop() {
    //emission salve US de 10µs
    digitalWrite(DOUT_TRIGGER, LOW);
    delayMicroseconds(2);
    digitalWrite(DOUT_TRIGGER, HIGH);
    delayMicroseconds(10);
    digitalWrite(DOUT_TRIGGER, LOW);

    //reception echo et calcul de la distance
    distance= pulseIn(DIN_ECHO, HIGH) /58.8235;
    if (distance>400){
        lcd.init();
        lcd.setCursor(0,0);
        lcd.print("echec de la ");
        lcd.setCursor(0,1);
        lcd.print("mesure");
        delay(800);
    }
    else{
        lcd.init();
        lcd.setCursor(0,0);
        lcd.print("distance");
        lcd.setCursor(0,1);
        lcd.print(distance);
        lcd.print("cm");
        delay(2000);
    }
}

![montage_lcd](telemetre_lcd.jpg)
